In [1]:
import numpy as np
import pandas as pd
import math


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_pickle('/content/drive/MyDrive/neuronske/data/full_sequences.pkl')
data.head()

,filename,mfccs,event_label,combine_squence_per_filename_and_event
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Dog,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Electric_shaver_toothbrush,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1238.wav,"[[-401.96198, -400.9539, -411.34525, -405.876,...",Blender,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
training = data[data['event_label'] == 'Blender']

X = np.array(training.mfccs.tolist())
y = np.array(training.combine_squence_per_filename_and_event.tolist())

X = np.einsum('ijk->ikj', X)

# indices = np.arange(X.shape[0])

# # split the dataset 
# from sklearn.model_selection import train_test_split 
# X_train, X_test, y_train, y_test, I_train, I_test = train_test_split(X, y, indices, test_size=0.2, random_state = 42)
training_filenames = training.filename.unique()

In [ ]:
training_false = data[(data['event_label'] != 'Blender') & (~data['filename'].isin(training_filenames))]
X_false = np.array(training_false.mfccs.tolist())
y_false = np.array(training_false.combine_squence_per_filename_and_event.tolist())

X_false = np.einsum('ijk->ikj', X_false)

In [ ]:
shape_of_y_false = y_false.shape
y_false = np.asarray([[0]*shape_of_y_false[1]]*shape_of_y_false[0])
y_false.shape

(2942, 431)

In [ ]:
Y_new = np.concatenate((y, y_false), axis=0)
print(Y_new.shape)
X_new = np.concatenate((X, X_false), axis=0)
print(X_new.shape)
X = X_new
y = Y_new

(3378, 431)
(3378, 431, 13)


In [ ]:
# split the dataset 
from sklearn.model_selection import train_test_split 
indices = np.arange(X.shape[0])
X_train, X_test, y_train, y_test, I_train, I_test = train_test_split(X, y, indices, test_size=0.2, random_state = 42)

In [ ]:
X_train.shape

(2702, 431, 13)

In [ ]:
# print(X_train.shape)
# B = np.einsum('ijk->ikj', X_train)
# print(B.shape)
# X_train = B
# print(X_test.shape)
# B = np.einsum('ijk->ikj', X_test)
# print(B.shape)
# X_test = B

(313, 13, 431)
(313, 431, 13)
(79, 13, 431)
(79, 431, 13)


.

.

. **GRU**

.

.



In [4]:
input_seq_len = 431
num_of_features = 13
# output_len = len(data)

batch_size = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dim = 128  # Latent dimensionality of the encoding space.


In [5]:
# Define architecture of autoencoder model. 
import tensorflow as tf
from tensorflow import keras

def create_encoder_decoder(input_seq_len = 431, num_of_features = 13):
  # Define an input sequence and process it.

  encoder_inputs = keras.Input(shape=(input_seq_len, num_of_features))
  encoder = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True)
  encoder_outputs, state_h = encoder(encoder_inputs)

  encoder_states = [state_h]
  # repeat_vector = keras.layers.RepeatVector(input_seq_len)(encoder_outputs)
  # Set up the decoder, using `encoder_states` as initial state.
  # decoder_inputs = keras.Input(shape=(25, 773))

  # We set up our decoder to return full output sequences.
  decoder_gru = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True)
  decoder_outputs, _ = decoder_gru(encoder_outputs, initial_state=encoder_states)
  decoder_dense = keras.layers.Dense(1, activation="sigmoid")
  decoder_outputs = decoder_dense(decoder_outputs)
  return encoder_inputs, decoder_outputs
# Define the model that should learn to replicate inputs to outputs.


In [ ]:
encoder_inputs, decoder_outputs = create_encoder_decoder()
model = keras.Model([encoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 431, 13)]    0                                            
__________________________________________________________________________________________________
gru (GRU)                       [(None, 431, 128), ( 54912       input_1[0][0]                    
__________________________________________________________________________________________________
gru_1 (GRU)                     [(None, 431, 128), ( 99072       gru[0][0]                        
                                                                 gru[0][1]                        
__________________________________________________________________________________________________
dense (Dense)                   (None, 431, 1)       129         gru_1[0][0]                  

In [ ]:
# Compile & train the model. For 100 epochs, 20k patients with 25 sequences each and with sequence with 773 features ~ 2.33 hours. 
# z was data_no_observations, x was array of y[0:18000]
model.compile(
    optimizer="adam", loss=tf.keras.losses.MeanSquaredError(), metrics=["binary_accuracy"]
)
model.fit(
    [X_train],
    [y_train],
    batch_size=batch_size,
    epochs=150,
    validation_split=0.2,
)
# Save model
# model.save("s2s")

Epoch 1/150
34/34 [==============================] - 11s 67ms/step - loss: 0.0607 - binary_accuracy: 0.9255 - val_loss: 0.0294 - val_binary_accuracy: 0.9706
Epoch 2/150
34/34 [==============================] - 2s 48ms/step - loss: 0.0413 - binary_accuracy: 0.9587 - val_loss: 0.0294 - val_binary_accuracy: 0.9706
Epoch 3/150
34/34 [==============================] - 2s 45ms/step - loss: 0.0413 - binary_accuracy: 0.9587 - val_loss: 0.0294 - val_binary_accuracy: 0.9706
Epoch 4/150
34/34 [==============================] - 2s 46ms/step - loss: 0.0413 - binary_accuracy: 0.9587 - val_loss: 0.0294 - val_binary_accuracy: 0.9706
Epoch 5/150
34/34 [==============================] - 2s 46ms/step - loss: 0.0412 - binary_accuracy: 0.9587 - val_loss: 0.0292 - val_binary_accuracy: 0.9706
Epoch 6/150
34/34 [==============================] - 2s 45ms/step - loss: 0.0377 - binary_accuracy: 0.9586 - val_loss: 0.0266 - val_binary_accuracy: 0.9706
Epoch 7/150
34/34 [==============================] - 2s 46ms/st

In [ ]:
predicted = model.predict([np.array([X_test[0]])])

print(np.round(np.concatenate(predicted[0], axis=0), 0).astype(int))
print(y_test[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred = (y_pred > 0.5)


11/11 [==============================] - 1s 30ms/step


In [6]:
# USED ONLY FOR TESTING
# NOT GOOD FOR SEQ TO SEQ, IT IS NOT JUST CLASSIFICATION
# IMPLEMENT WHAT THEY DID IN TASK WHERE WE GOT THE IDEA

# CONCLUSIONS: TOO MUCH FALSE NEGATIVES - BAD RECALL
#              BIGGER LATENT_TIM -> BETTER RESULTS -- LAT_DIM-F1 : 13-68, 64-70 256-73
def simple_evaluation(y_pred, y_test):
  tp = 0
  tn = 0
  fp = 0
  fn = 0

  for i, pred_arr in enumerate(y_pred):
    for j, pred_value in enumerate(pred_arr):
      predicted = pred_value
      real = y_test[i][j]
      if predicted and real:
        tp = tp + 1
      elif not(predicted) and real:
        fn = fn + 1
      elif predicted and not(real):
        fp = fp + 1
      elif not(predicted) and not(real):
        tn = tn + 1
    # break

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 2 * precision * recall / (precision + recall)

  print("PRECISION: " + str(precision))
  print("RECALL: " + str(recall))
  print("F1: " + str(f1))
  print("-"*20)
  print("tp: " + str(tp))
  print("tn: " + str(tn))
  print("fp: " + str(fp))
  print("fn: " + str(fn))


In [ ]:
simple_evaluation(y_pred, y_test)

PRECISION: 0.7740529966210208
RECALL: 0.6071279118426559
F1: 0.680503439649781
--------------------
tp: 8705
tn: 274477
fp: 2541
fn: 5633


.

.

. **RESULTS PREPARATION**

.

.



In [7]:
!pip install sed_eval


     |████████████████████████████████| 2.1MB 9.3MB/s 
  Created wheel for sed-eval: filename=sed_eval-0.2.1-cp37-none-any.whl size=26124 sha256=cfabca237101ed2f5c660dcab002275a5b45c08a7c0de6fa76765e7bda604e82
  Stored in directory: /root/.cache/pip/wheels/3c/72/5d/5fc941f98c583ce9d8adee7f13e24b46459aeded4125f9c369
  Created wheel for dcase-util: filename=dcase_util-0.2.18-cp37-none-any.whl size=2147233 sha256=e11fb17dba3d87faef614e2f972aaf3666499ed7ecd935f9c695c0277ae2eecb
  Stored in directory: /root/.cache/pip/wheels/d1/5d/9e/c94cd8bfb80541c73d549c734cafcd3de994dbb117a6485c81
Successfully built sed-eval dcase-util


In [8]:
events = pd.read_csv("/content/drive/MyDrive/neuronske/data/Synthetic_dataset/synthetic_dataset.csv", delimiter="\t") 
events.head()

,filename,onset,offset,event_label
0,10.wav,0.287159,1.057712,Alarm_bell_ringing
1,10.wav,1.717124,1.967124,Alarm_bell_ringing
2,10.wav,5.049573,6.013632,Alarm_bell_ringing
3,10.wav,7.240901,8.492352,Speech
4,1000.wav,4.352958,4.664092,Alarm_bell_ringing


In [9]:
import sed_eval
import dcase_util

def return_onsets_and_offsets_for_sequence(sequence, duration_seconds=10):
  single_tick_in_seconds = duration_seconds / len(sequence)
  onsets = []
  offsets = []

  error_allowance = 5
  error_counter = 0
  in_positive_sequence = 0
  
  for index, tick in enumerate(sequence):
    if tick and not(in_positive_sequence):
      in_positive_sequence = 1
      onsets.append(index * single_tick_in_seconds)
    elif not(tick) and in_positive_sequence:
      if error_counter >= error_allowance:
        error_counter = 0
        in_positive_sequence = 0
        offsets.append((index - error_allowance) * single_tick_in_seconds)
      else:
        error_counter = error_counter + 1
    elif tick and in_positive_sequence and error_counter:
      error_counter = 0
  if in_positive_sequence:
    offsets.append((index) * single_tick_in_seconds)

  return onsets, offsets

def prepare_predicted_sequences_for_evaluation_and_evaluate(event_label, y_pred, I_test, training_data, events_data, return_prepared_data_flag = 0):
  references = []
  estimations = []

  for i, y_predicted in enumerate(y_pred):
    file_index = I_test[i]
    file_name = training_data.iloc[file_index]['filename']

    onsets, offsets = return_onsets_and_offsets_for_sequence(y_predicted, 10)

    for j, onset in enumerate(onsets):
      estimations.append({
          'event_label': event_label,
          'event_onset': onset,
          'event_offset': offsets[j],
          'file': file_name
      })

    reference_events = events_data[(events_data['filename'] == file_name) & (events_data['event_label'] == event_label)]
    for ref_event in np.asarray(reference_events):
      references.append({
          'event_label': event_label,
          'event_onset': ref_event[1],
          'event_offset': ref_event[2],
          'file': file_name
      })

  evaluate_and_print_results(references, estimations)

  if return_prepared_data_flag:
    return references, estimations


def evaluate_and_print_results(references, estimations):
  reference_event_list = dcase_util.containers.MetaDataContainer(
      references
  )

  estimated_event_list = dcase_util.containers.MetaDataContainer(
      estimations
  )

  segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
      event_label_list=reference_event_list.unique_event_labels,
      time_resolution=1.0
  )
  event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
      event_label_list=reference_event_list.unique_event_labels,
      t_collar=0.250,
      evaluate_onset=True,
      evaluate_offset=True,
      percentage_of_length=0.5
  )

  for filename in reference_event_list.unique_files:
      reference_event_list_for_current_file = reference_event_list.filter(
          filename=filename
      )

      estimated_event_list_for_current_file = estimated_event_list.filter(
          filename=filename
      )

      segment_based_metrics.evaluate(
          reference_event_list=reference_event_list_for_current_file,
          estimated_event_list=estimated_event_list_for_current_file
      )

      event_based_metrics.evaluate(
          reference_event_list=reference_event_list_for_current_file,
          estimated_event_list=estimated_event_list_for_current_file
      )

  # Get only certain metrics
  overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
  print("Accuracy:", overall_segment_based_metrics['accuracy']['accuracy'])

  # Or print all metrics as reports
  print(segment_based_metrics)
  print(event_based_metrics)

In [10]:
def combine_predicted_to_original(y_pred, seconds, sequence_len):
  ticks_per_chosen_second = math.ceil(sequence_len / (10 / seconds))
  num_of_parts_of_sequence = math.ceil(sequence_len / ticks_per_chosen_second)
  y_pred_combined = []
  # Num of arrays at the end
  final_size = math.floor(y_pred.shape[0] / num_of_parts_of_sequence)

  print(num_of_parts_of_sequence)
  print(final_size)

  for i in range(0, final_size):
    current_arr = np.array([]).reshape(0,y_pred[0].shape[1])
    for j in range(0, num_of_parts_of_sequence):
      current_arr = np.concatenate((current_arr, y_pred[i * num_of_parts_of_sequence + j])).astype(int)
    y_pred_combined.append(current_arr)
  return y_pred_combined

.

.

. **ANSAMBLE SPLITTED PER SECOND**

.

.



In [11]:
data = pd.read_pickle('/content/drive/MyDrive/neuronske/data/full_sequences.pkl')
data.head()

,filename,mfccs,event_label,combine_squence_per_filename_and_event
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Dog,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Electric_shaver_toothbrush,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1238.wav,"[[-401.96198, -400.9539, -411.34525, -405.876,...",Blender,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
def chunks(lst, n):
  """Yield successive n-sized chunks from lst."""
  return ([lst[i:i + n] for i in range(0, len(lst), n)])


def split_sequences_into_seconds(X, y, seconds = 1):
  sequence_len = X.shape[1]
  ticks_per_chosen_second = math.ceil(sequence_len / (10 / seconds))
  num_of_parts_of_sequence = math.ceil(sequence_len / ticks_per_chosen_second)

  X_new = []
  y_new = []
  for i, sequence in enumerate(X):
    X_splits = chunks(sequence, ticks_per_chosen_second)
    y_splits = chunks(y[i], ticks_per_chosen_second)
    x_false_instances = np.array([]).reshape(0,X_splits[0].shape[1])
    for j, X_split in enumerate(X_splits):
      X_temp = X_split
      y_temp = y_splits[j]
      if len(x_false_instances) < ticks_per_chosen_second:
        false_indices = np.where(~y_temp)
        x_false_instances = np.append(x_false_instances, X_split[false_indices], axis=0)

      diff = ticks_per_chosen_second - len(X_temp)
      if diff != 0:
        y_temp = np.append(y_temp, [0] * diff)
        X_temp = np.append(X_temp, x_false_instances[0: diff], axis=0)
        
      X_new.append(X_temp)
      y_new.append(y_temp)


  return np.asarray(X_new), np.asarray(y_new)

In [ ]:
X_splitted, y_splitted = split_sequences_into_seconds(X, y, 1)

print(X.shape)
print(X_splitted.shape)
print(y.shape)
print(y_splitted.shape)

(196, 431, 13)
(1960, 44, 13)
(196, 431)
(1960, 44)


In [13]:
unique_events = np.sort(data['event_label'].unique())
print(unique_events)

['Alarm_bell_ringing' 'Blender' 'Cat' 'Dishes' 'Dog'
 'Electric_shaver_toothbrush' 'Frying' 'Running_water' 'Speech'
 'Vacuum_cleaner']


In [16]:
from sklearn.model_selection import train_test_split 

for index, event in enumerate(unique_events):
  training = data[data['event_label'] == event]

  X = np.array(training.mfccs.tolist())
  y = np.array(training.combine_squence_per_filename_and_event.tolist())

  X = np.einsum('ijk->ikj', X)
  # TO USE ALL DATA
  # training_filenames = training.filename.unique()

  # training_false = data[(data['event_label'] != event) & (~data['filename'].isin(training_filenames))]

  # X_false = np.array(training_false.mfccs.tolist())
  # y_false = np.array(training_false.combine_squence_per_filename_and_event.tolist())
  # X_false = np.einsum('ijk->ikj', X_false)

  # shape_of_y_false = y_false.shape
  # y_false = np.asarray([[0]*shape_of_y_false[1]]*shape_of_y_false[0])

  # y = np.concatenate((y, y_false), axis=0)
  # X = np.concatenate((X, X_false), axis=0)
  #


  # TO SPLIT SEQUENCES
  X, y = split_sequences_into_seconds(X, y, 5)
  #

  
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

  print("-"*80)
  print(str(index) + " ------ " + event + " " + str(len(X)))

  encoder_inputs, decoder_outputs = create_encoder_decoder(X.shape[1], X.shape[2])
  model = keras.Model([encoder_inputs], decoder_outputs)
  model.compile(
    optimizer="adam", loss=tf.keras.losses.MeanSquaredError(), metrics=["binary_accuracy"]
  )

  checkpoint_filepath = '/content/drive/MyDrive/neuronske/models/checkpoint/seq_to_seq'
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_binary_accuracy',
    mode='max',
    save_best_only=True)

  model.fit(
      [X_train],
      [y_train],
      batch_size=batch_size,
      epochs=150,
      validation_split=0.2, 
      callbacks=[model_checkpoint_callback]
  )
  model.load_weights(checkpoint_filepath)
  model.save('/content/drive/MyDrive/neuronske/models/' + str(event) + '.h5')
  print("-"*80)


--------------------------------------------------------------------------------
0 ------ Alarm_bell_ringing 784
Epoch 1/150
8/8 [==============================] - 3s 114ms/step - loss: 0.1985 - binary_accuracy: 0.7429 - val_loss: 0.1848 - val_binary_accuracy: 0.8027
Epoch 2/150
8/8 [==============================] - 0s 31ms/step - loss: 0.1821 - binary_accuracy: 0.7969 - val_loss: 0.1516 - val_binary_accuracy: 0.8079
Epoch 3/150
8/8 [==============================] - 0s 28ms/step - loss: 0.1466 - binary_accuracy: 0.8094 - val_loss: 0.1298 - val_binary_accuracy: 0.8256
Epoch 4/150
8/8 [==============================] - 0s 27ms/step - loss: 0.1314 - binary_accuracy: 0.8196 - val_loss: 0.1202 - val_binary_accuracy: 0.8385
Epoch 5/150
8/8 [==============================] - 0s 28ms/step - loss: 0.1167 - binary_accuracy: 0.8553 - val_loss: 0.1084 - val_binary_accuracy: 0.8500
Epoch 6/150
8/8 [==============================] - 0s 27ms/step - loss: 0.1064 - binary_accuracy: 0.8604 - val_loss:

In [17]:
# RESULTS LOOK OK
# ANALYZE HOW IT BEHAVES WITH OVERLAPING EVENTS
combined_references = []
combined_estimations = []

for index, event in enumerate(unique_events):
  training = data[data['event_label'] == event]

  X = np.array(training.mfccs.tolist())
  y = np.array(training.combine_squence_per_filename_and_event.tolist())

  X = np.einsum('ijk->ikj', X)

  # TO USE ALL DATA
  # training_filenames = training.filename.unique()
  # training_false = data[(data['event_label'] != event) & (~data['filename'].isin(training_filenames))]
  # X_false = np.array(training_false.mfccs.tolist())
  # y_false = np.array(training_false.combine_squence_per_filename_and_event.tolist())
  # X_false = np.einsum('ijk->ikj', X_false)
  # shape_of_y_false = y_false.shape
  # y_false = np.asarray([[0]*shape_of_y_false[1]]*shape_of_y_false[0])
  # y = np.concatenate((y, y_false), axis=0)
  # X = np.concatenate((X, X_false), axis=0)
  # training = training.append(training_false, ignore_index=True)
  # 

  indices = np.arange(X.shape[0])
  X_train, X_test, y_train, y_test, I_train, I_test = train_test_split(X, y, indices, test_size=0.2, random_state = 42)

  X_train, y_train = split_sequences_into_seconds(X_train, y_train, 5)
  X_test, y_test = split_sequences_into_seconds(X_test, y_test, 5)


  print("-"*80)
  print(str(index) + " ------ " + event + " " + str(len(X)))
  model = tf.keras.models.load_model('/content/drive/MyDrive/neuronske/models/' + str(event) + '.h5', compile=True)
  y_pred = model.predict(X_test, batch_size=64, verbose=1)
  y_pred = (y_pred > 0.5)
  simple_evaluation(y_pred, y_test)
  print("-"*80)
  print(y_pred.shape)
  y_pred = combine_predicted_to_original(y_pred, 5, 431)
  # combined_test_y = combine_predicted_to_original(y_test, 1, 431)

  references, estimations = prepare_predicted_sequences_for_evaluation_and_evaluate(event, y_pred, I_test, training, events, 1)
  combined_references.extend(references)
  combined_estimations.extend(estimations)
  print("-"*80)


print("-"*80)
print("-"*80)

print("FINAL EVALUATION: ")
print("-"*80)
evaluate_and_print_results(combined_references, combined_estimations)
print("-"*80)

#         transfer      no-transfer          1 sec      2 secs    0.5 sec
# alarm   0,75          0,67                 0,77       0,74      0,75
# blender 0,87          0,88                 0,93       0,91      0,93
# cat     0,71          0,77                 0,78       0,75      0,76
# dishes  0,34          0,41                 0,54       0,50      0,51
# dog     0,59          0,65                 0,77       0,75      0,75
# shaver  0,94          0,94                 0,97       0,96      0,98
# frying  0,87          0,88                 0,90       0,92      0,93
# water   0,85          0,86                 0,88       0,88      0,89
# speech  0,75          0,76                 0,81       0,79      0,77
# vacuum  0,91          0,92                 0,94       0,92      0,94

--------------------------------------------------------------------------------
0 ------ Alarm_bell_ringing 392
3/3 [==============================] - 0s 18ms/step
PRECISION: 0.8862603581929965
RECALL: 0.8867344209681733
F1: 0.8864973262032085
--------------------
tp: 6631
tn: 25799
fp: 851
fn: 847
--------------------------------------------------------------------------------
(158, 216, 1)
2
79
Accuracy: 0.8979289940828402
Segment based metrics
  Evaluated length                  : 647.47 sec
  Evaluated files                   : 79 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 89.06 %
    Precision                       : 85.93 %
    Recall                          : 92.43 %
  Error rate
    Error rate (ER)                 : 0.23 
    Substitution rate               : 0.00 
    Deletion rate                   : 0.08 
    Insertion rate                  : 0.15 
  Accuracy
    Sensitivity          

.

.

. **ANSAMBLE ON 10 SECONDS SEQUENCE**

.

.



In [ ]:
data = pd.read_pickle('/content/drive/MyDrive/neuronske/data/full_sequences.pkl')
data.head()

,filename,mfccs,event_label,combine_squence_per_filename_and_event
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Dog,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
0,2282.wav,"[[-659.70654, -618.7009, -581.4676, -577.1263,...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Electric_shaver_toothbrush,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2719.wav,"[[-584.7399, -515.89514, -473.47324, -464.3819...",Speech,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1238.wav,"[[-401.96198, -400.9539, -411.34525, -405.876,...",Blender,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
unique_events = np.sort(data['event_label'].unique())
print(unique_events)

['Alarm_bell_ringing' 'Blender' 'Cat' 'Dishes' 'Dog'
 'Electric_shaver_toothbrush' 'Frying' 'Running_water' 'Speech'
 'Vacuum_cleaner']


In [ ]:
from sklearn.model_selection import train_test_split 

for index, event in enumerate(unique_events):
  training = data[data['event_label'] == event]

  X = np.array(training.mfccs.tolist())
  y = np.array(training.combine_squence_per_filename_and_event.tolist())

  X = np.einsum('ijk->ikj', X)


  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

  print("-"*80)
  print(str(index) + " ------ " + event + " " + str(len(X)))

  encoder_inputs, decoder_outputs = create_encoder_decoder(X.shape[1], X.shape[2])
  model = keras.Model([encoder_inputs], decoder_outputs)
  model.compile(
    optimizer="adam", loss=tf.keras.losses.MeanSquaredError(), metrics=["binary_accuracy"]
  )
  model.fit(
      [X_train],
      [y_train],
      batch_size=batch_size,
      epochs=150,
      validation_split=0.2,
  )
  model.save('/content/drive/MyDrive/neuronske/models/' + str(event) + '_10_secs_seq.h5')
  print("-"*80)


--------------------------------------------------------------------------------
0 ------ Alarm_bell_ringing 392
Epoch 1/150
4/4 [==============================] - 3s 266ms/step - loss: 0.2134 - binary_accuracy: 0.6517 - val_loss: 0.1571 - val_binary_accuracy: 0.8268
Epoch 2/150
4/4 [==============================] - 0s 54ms/step - loss: 0.1861 - binary_accuracy: 0.7961 - val_loss: 0.1606 - val_binary_accuracy: 0.8268
Epoch 3/150
4/4 [==============================] - 0s 48ms/step - loss: 0.1828 - binary_accuracy: 0.7961 - val_loss: 0.1517 - val_binary_accuracy: 0.8268
Epoch 4/150
4/4 [==============================] - 0s 50ms/step - loss: 0.1609 - binary_accuracy: 0.7976 - val_loss: 0.1421 - val_binary_accuracy: 0.8220
Epoch 5/150
4/4 [==============================] - 0s 50ms/step - loss: 0.1465 - binary_accuracy: 0.8044 - val_loss: 0.1557 - val_binary_accuracy: 0.8024
Epoch 6/150
4/4 [==============================] - 0s 48ms/step - loss: 0.1375 - binary_accuracy: 0.8119 - val_loss:

In [ ]:
# RESULTS LOOK OK
# ANALYZE HOW IT BEHAVES WITH OVERLAPING EVENTS
combined_references = []
combined_estimations = []

for index, event in enumerate(unique_events):
  training = data[data['event_label'] == event]

  X = np.array(training.mfccs.tolist())
  y = np.array(training.combine_squence_per_filename_and_event.tolist())

  X = np.einsum('ijk->ikj', X)

  indices = np.arange(X.shape[0])
  X_train, X_test, y_train, y_test, I_train, I_test = train_test_split(X, y, indices, test_size=0.2, random_state = 42)


  print("-"*80)
  print(str(index) + " ------ " + event + " " + str(len(X)))
  model = tf.keras.models.load_model('/content/drive/MyDrive/neuronske/models/' + str(event) + '_10_secs_seq.h5', compile=True)
  y_pred = model.predict(X_test, batch_size=64, verbose=1)
  y_pred = (y_pred > 0.5)
  simple_evaluation(y_pred, y_test)
  print("-"*80)
  print(y_pred.shape)

  references, estimations = prepare_predicted_sequences_for_evaluation_and_evaluate(event, y_pred, I_test, training, events, 1)
  combined_references.extend(references)
  combined_estimations.extend(estimations)
  print("-"*80)


print("-"*80)
print("-"*80)

print("FINAL EVALUATION: ")
print("-"*80)
evaluate_and_print_results(combined_references, combined_estimations)
print("-"*80)


--------------------------------------------------------------------------------
0 ------ Alarm_bell_ringing 392
2/2 [==============================] - 0s 25ms/step
PRECISION: 0.7568627450980392
RECALL: 0.6710350361059106
F1: 0.7113694357811171
--------------------
tp: 5018
tn: 24959
fp: 1612
fn: 2460
--------------------------------------------------------------------------------
(79, 431, 1)
Accuracy: 0.8168812589413448
Segment based metrics
  Evaluated length                  : 673.40 sec
  Evaluated files                   : 79 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 79.68 %
    Precision                       : 76.99 %
    Recall                          : 82.57 %
  Error rate
    Error rate (ER)                 : 0.42 
    Substitution rate               : 0.00 
    Deletion rate                   : 0.17 
    Insertion rate                  : 0.25 
  Accuracy
    Sensitivity              